In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#expands all columns in datafrae
pd.options.display.max_columns = None

#gets rid of SettingWithCopyWarning:
pd.options.mode.chained_assignment = None

In [2]:
#import the data
data = pd.read_json("data/data.json")

In [3]:
data.head(2)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC


In [4]:
#number of features
print ("There are {} features.".format(data.shape[1]))

There are 44 features.


In [5]:
#number of datapoints
print ("there are {} datapoints.".format(data.shape[0]))

there are 14337 datapoints.


### Create Label

In [6]:
data.acct_type.unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

In [7]:
#add a new column called "Fraud"
#True if any acct_type field contains the word fraud
#False if it does not contain it
data["Fraud"] = data.acct_type.str.contains("fraud", regex = True)

In [8]:
data.head(2)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,Fraud
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,True
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC,False


In [9]:
#this will be our target
label = data["Fraud"]

## Split our data into train and test data

In [10]:
X = data.drop("Fraud", axis = 1)
y = data["Fraud"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [12]:
events = X_train
actual = y_train

## Start EDA

In [13]:
events.describe()

,approx_payout_date,body_length,channels,delivery_method,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,name_length,num_order,num_payouts,object_id,org_facebook,org_twitter,sale_duration,sale_duration2,show_map,user_age,user_created,user_type,venue_latitude,venue_longitude
count,9.605000e+03,9605.00000,9605.000000,9593.000000,9.605000e+03,9.605000e+03,9.536000e+03,9.605000e+03,9605.000000,9605.000000,9605.000000,5959.000000,9605.000000,9605.000000,9605.000000,9605.000000,9.605000e+03,9562.000000,9562.000000,9496.000000,9605.000000,9605.000000,9605.000000,9.605000e+03,9605.000000,8855.000000,8855.000000
mean,1.351181e+09,3664.60760,6.208433,0.439591,1.346061e+09,1.350749e+09,1.340760e+09,1.350447e+09,0.125976,2329.154699,0.072462,0.202047,0.843103,41.352108,27.237168,33.706091,4.553987e+06,8.046852,4.345744,46.904802,50.647892,0.845601,373.858719,1.313760e+09,2.707236,35.189824,-58.590085
std,2.404474e+07,5795.97955,3.863545,0.590200,2.406688e+07,2.404474e+07,9.016100e+07,2.402890e+07,0.331840,8340.768318,0.259265,0.401561,0.363723,22.793902,90.046248,125.547252,1.989359e+06,11.781919,6.031374,55.287367,59.747577,0.361349,419.236498,4.183621e+07,1.494219,20.442063,67.306660
min,1.173074e+09,0.00000,0.000000,0.000000,1.171223e+09,1.172642e+09,0.000000e+00,1.172633e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.812900e+04,0.000000,0.000000,-1094.000000,-1094.000000,0.000000,0.000000,1.011210e+09,1.000000,-45.045208,-159.390183
25%,1.334534e+09,740.00000,5.000000,0.000000,1.329872e+09,1.334102e+09,1.329800e+09,1.333688e+09,0.000000,117.740000,0.000000,0.000000,1.000000,26.000000,2.000000,0.000000,2.914355e+06,0.000000,0.000000,16.000000,17.000000,1.000000,13.000000,1.286843e+09,1.000000,34.000355,-96.805805
50%,1.356140e+09,1965.00000,6.000000,0.000000,1.351532e+09,1.355708e+09,1.351557e+09,1.355461e+09,0.000000,429.800000,0.000000,0.000000,1.000000,38.000000,8.000000,2.000000,4.625808e+06,0.000000,0.000000,32.000000,35.000000,1.000000,243.000000,1.317212e+09,3.000000,39.993109,-79.396375
75%,1.370844e+09,4257.00000,8.000000,1.000000,1.365465e+09,1.370412e+09,1.365621e+09,1.370106e+09,0.000000,1524.150000,0.000000,0.000000,1.000000,52.000000,23.000000,13.000000,6.106807e+06,14.000000,11.000000,59.000000,64.000000,1.000000,597.000000,1.347389e+09,3.000000,44.949016,-2.935394
max,1.523488e+09,65535.00000,13.000000,3.000000,1.382012e+09,1.523056e+09,1.381544e+09,1.418850e+09,1.000000,225610.000000,1.000000,1.000000,1.000000,254.000000,2000.000000,1858.000000,8.792893e+06,91.000000,34.000000,980.000000,980.000000,1.000000,3794.000000,1.381543e+09,103.000000,64.844022,176.168280


In [14]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9605 entries, 13845 to 7742
Data columns (total 44 columns):
acct_type             9605 non-null object
approx_payout_date    9605 non-null int64
body_length           9605 non-null int64
channels              9605 non-null int64
country               9544 non-null object
currency              9605 non-null object
delivery_method       9593 non-null float64
description           9605 non-null object
email_domain          9605 non-null object
event_created         9605 non-null int64
event_end             9605 non-null int64
event_published       9536 non-null float64
event_start           9605 non-null int64
fb_published          9605 non-null int64
gts                   9605 non-null float64
has_analytics         9605 non-null int64
has_header            5959 non-null float64
has_logo              9605 non-null int64
listed                9605 non-null object
name                  9605 non-null object
name_length           9605 non-nul

In [15]:
#checking to see if any features of NaN values
#true indicates the feature has NaN values
events.isna().any()

acct_type             False
approx_payout_date    False
body_length           False
channels              False
country                True
currency              False
delivery_method        True
description           False
email_domain          False
event_created         False
event_end             False
event_published        True
event_start           False
fb_published          False
gts                   False
has_analytics         False
has_header             True
has_logo              False
listed                False
name                  False
name_length           False
num_order             False
num_payouts           False
object_id             False
org_desc              False
org_facebook           True
org_name              False
org_twitter            True
payee_name            False
payout_type           False
previous_payouts      False
sale_duration          True
sale_duration2        False
show_map              False
ticket_types          False
user_age            

##### body_length

In [16]:
events.body_length.value_counts()

0        565
870       21
108       18
1615      16
3684      11
131        9
159        8
218        8
145        8
4076       8
1481       8
203        7
589        7
155        7
460        7
115        7
1069       7
955        7
928        7
469        7
834        7
1351       7
138        7
1456       7
215        7
277        7
407        7
241        7
201        7
256        7
        ... 
6917       1
15268      1
5025       1
7128       1
20113      1
2751       1
2775       1
3106       1
10947      1
4804       1
6853       1
10955      1
9231       1
6861       1
4820       1
3313       1
13317      1
5121       1
987        1
13016      1
3066       1
734        1
3058       1
5105       1
4853       1
1003       1
2733       1
2791       1
15081      1
4174       1
Name: body_length, Length: 5256, dtype: int64

##### currency types

In [17]:
#what currencies were used?
currencies = events.currency.unique()
currencies

array(['AUD', 'CAD', 'USD', 'GBP', 'EUR', 'NZD', 'MXN'], dtype=object)

##### description feature

In [23]:
events.org_desc.iloc[0]

'<p style="margin: 0.0px 0.0px 0.0px 0.0px; font: 14.0px Palatino; color: #5a3e51;"><span style="font-size: small; color: #000000;"><span class="Apple-style-span" style="font-family: arial, helvetica, sans-serif;"><span class="Apple-style-span" style="line-height: 21px; font-family: arial, helvetica, sans-serif; font-size: small; color: #000000;">Hub Melbourne is a professional member community that DRIVES innovation through collaboration.&nbsp;</span></span></span><span style="font-size: small; color: #000000;"><span class="Apple-style-span" style="font-family: arial, helvetica, sans-serif;"><span class="Apple-style-span" style="line-height: 21px; font-family: arial, helvetica, sans-serif; font-size: small; color: #000000;">We do this by providing diverse individuals and organisations with a thriving space for co-working, meetings and events.&nbsp;We also&nbsp;</span>host learning and collaboration experiences that create value for our members, their projects, and the community.&nbsp;

In [41]:
from bs4 import BeautifulSoup
import re

In [44]:
# soup = BeautifulSoup(events.org_desc.iloc[0]).text.replace("\xa0", " ").replace("\n", " ").lower()


In [60]:
soup = BeautifulSoup(events.org_desc.iloc[0]).text
re.sub('\xa0', " ", soup)

'Hub Melbourne is a professional member community that DRIVES innovation through collaboration. We do this by providing diverse individuals and organisations with a thriving space for co-working, meetings and events. We also host learning and collaboration experiences that create value for our members, their projects, and the community. Events are open to both members and non-members with members receiving free or discounted entry. To find out how you or your organisation can benefit from membership, please click here.\n \nTo use the creative, innovative spaces at Hub Melbourne for your next meeting and event, please click here. If you would like to know more about using Hub Melbourne as your workspace please click here.\n \n \n \n '